In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

Wed Oct 21 12:38:56 PDT 2020


# Extract, transform, and "load" data from Facebook Data for Good USA survey

This code requires access to the IHME cluster to run; we have included it for completeness.

In [2]:
def my_etl(df):
    t = pd.DataFrame()

    t['age_group'] = df.D2
    t['male'] = (df.D1 == 1).astype(int)
    
    t['test_last_14_days'] = (df.B10 == 1).astype(int)
    t['test_positive'] = (df.B10a == 1).astype(int)
    t['test_negative'] = (df.B10a == 2).astype(int)
    t['test_unknown'] = (df.B10a == 3).astype(int)
    t['test_reason'] = df.B10b
    t['test_required'] = df.B10b.astype(str).str.contains('4')
    
    t['hcw'] = (df.Q64.isin([4,5]))
    t = pd.concat([t, df.filter(regex='^[DQ]')], axis=1)

    t['work_outside_home'] = (df.D10 == 1).astype(int)

    t['weight'] = df.weight
    return t

In [3]:
# test ETL function on the most recent available data file

import glob
fnames = sorted(glob.glob('/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/'
                   'extracted_daily_w4/*.csv'), reverse=True)
fname = fnames[0]
print(fname)
df = pd.read_csv(fname, low_memory=False)
%time my_etl(df.iloc[:1_000])

/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_18_recordedby_2020_10_18.csv
CPU times: user 11.8 ms, sys: 0 ns, total: 11.8 ms
Wall time: 11.8 ms


,age_group,male,test_last_14_days,test_positive,test_negative,test_unknown,test_reason,test_required,hcw,D1,...,Q74,Q75,Q76,Q77,Q78,Q79,Q80,D10,work_outside_home,weight
0,7.0,0,0,0,0,0,NaN,False,False,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5974.699351
1,NaN,0,0,0,0,0,NaN,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9493.929210
2,NaN,0,0,0,0,0,NaN,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10117.309576


In [4]:
# parse date out of filenames to be able to record when survey was offered

def my_date(s):
    s1 = s.split('cvid_responses_')[1]
    s2 = s1.split('_recordedby_')[0]
    s3 = s2.replace('_', '-')
    return s3
# results_us['date'] = results_us.index.map(my_date)

In [5]:
%%time

# loop over all data files from wave 4 of survey
# and extract data from most recent file for each day

data = {}
for fname in fnames:
    print(fname)
    data_date = my_date(fname)
    if data_date in data.keys():
        print('already loaded data for', data_date)
    else:
        if data_date <= '2020-09-03':  # earlier waves of survey do not contain key questions for this analysis
            continue
#         print('loading data for', data_date)
        df_i = pd.read_csv(fname, low_memory=False)
        %time data[data_date] = my_etl(df_i)

#         print(my_date(fname), 'n_positive=', np.sum(data[data_date].test_positive),
#               'n_negative=', np.sum(data[data_date].test_negative))


/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_18_recordedby_2020_10_18.csv
CPU times: user 9.85 ms, sys: 847 µs, total: 10.7 ms
Wall time: 10.4 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_17_recordedby_2020_10_18.csv
CPU times: user 43 ms, sys: 1.79 ms, total: 44.8 ms
Wall time: 44.8 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_16_recordedby_2020_10_18.csv
CPU times: user 42.3 ms, sys: 6 ms, total: 48.3 ms
Wall time: 48.3 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_16_recordedby_2020_10_17.csv
already loaded data for 2020-10-16
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_16_recordedby_2020_10_16.csv
already loaded data for 2020-10-16
/i

CPU times: user 49.1 ms, sys: 976 µs, total: 50.1 ms
Wall time: 50.1 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_06_recordedby_2020_10_09.csv
already loaded data for 2020-10-06
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_06_recordedby_2020_10_08.csv
already loaded data for 2020-10-06
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_06_recordedby_2020_10_07.csv
already loaded data for 2020-10-06
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_05_recordedby_2020_10_09.csv
CPU times: user 53.8 ms, sys: 3.91 ms, total: 57.7 ms
Wall time: 57.7 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_10_05_recordedby_2020_10_08.csv
already loaded data for 2020-10-05


CPU times: user 46.1 ms, sys: 0 ns, total: 46.1 ms
Wall time: 46.1 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_25_recordedby_2020_09_28.csv
already loaded data for 2020-09-25
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_25_recordedby_2020_09_27.csv
already loaded data for 2020-09-25
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_25_recordedby_2020_09_26.csv
already loaded data for 2020-09-25
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_24_recordedby_2020_09_28.csv
CPU times: user 47.2 ms, sys: 0 ns, total: 47.2 ms
Wall time: 47.2 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_24_recordedby_2020_09_27.csv
already loaded data for 2020-09-24
/ihme

CPU times: user 57.9 ms, sys: 1.98 ms, total: 59.9 ms
Wall time: 59.9 ms
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_14_recordedby_2020_09_17.csv
already loaded data for 2020-09-14
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_14_recordedby_2020_09_16.csv
already loaded data for 2020-09-14
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_14_recordedby_2020_09_15.csv
already loaded data for 2020-09-14
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_14_recordedby_2020_09_14.csv
already loaded data for 2020-09-14
/ihme/limited_use/LIMITED_USE/PROJECT_FOLDERS/COVID19/SYMPTOM_SURVEY/US/extracted_daily_w4/cvid_responses_2020_09_13_recordedby_2020_09_17.csv
CPU times: user 56.7 ms, sys: 0 ns, total: 56.7 ms
Wall time: 56.7 ms
/i

In [6]:
for k in data.keys():
    data[k]['date'] = k

In [7]:
df_all = pd.concat(data.values())
df_all

,age_group,male,test_last_14_days,test_positive,test_negative,test_unknown,test_reason,test_required,hcw,D1,...,Q75,Q76,Q77,Q78,Q79,Q80,D10,work_outside_home,weight,date
0,7.0,0,0,0,0,0,NaN,False,False,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5974.699351,2020-10-18
1,NaN,0,0,0,0,0,NaN,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9493.929210,2020-10-18
2,NaN,0,0,0,0,0,NaN,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10117.309576,2020-10-18
0,5.0,0,0,0,0,0,NaN,False,False,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6965.666632,2020-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38116,NaN,0,0,0,0,0,NaN,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3759.360136,2020-09-04
38117,6.0,1,0,0,0,0,NaN,False,False,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9224.525681,2020-09-04
38118,2.0,1,0,0,0,0,NaN,False,False,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9493.309105,2020-09-04
38119,2.0,1,0,0,0,0,NaN,False,False,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9681.808153,2020-09-04


In [8]:
df_all.to_csv('data/fb_data_usa.csv')

In [9]:
df_all.shape

(1862697, 41)